### 2.2.3 標準インターフェイスとしての列


In [1]:
; ⽊を引数に取り、奇数の葉の⼆乗の合計
(define (square x)(* x x))

(define (sum-odd-squares tree)
  (cond ((null? tree) 0)
        ((not (pair? tree)) (if (odd? tree) (square tree) 0))
        (else (+ (sum-odd-squares (car tree)) (sum-odd-squares (cdr tree)))))
  )
(define x (list 1 (list 2 (list 3 4) 5) (list 6 7)))
(display x)
(newline)
(display (sum-odd-squares x))
(newline)

(1 (2 (3 4) 5) (6 7))
84


In [2]:
; フィボナッチ数列の偶数のもののリスト
(define (fib n)
  (cond ((= n 0) 0)
        ((= n 1) 1) 
        (else (+ (fib (- n 1)) (fib (- n 2))))
        )
  )

(define (even-fibs n)
  (define (next k)
    (if (> k n) '()
        (let ((f (fib k)))
          (if (even? f) (cons f (next (+ k 1)))
              (next (+ k 1))))
      )
    )
  (next 0)
  )

;(define (map proc items)
;  (if (null? items) '()
;      (cons (proc (car items)) (map proc (cdr items)))))
; ->定義済み

(define (make-list n m)
  (if (> n m) (list)
      (append (list n) (make-list (+ n 1) m))
    )
  )

;(display (map fib (list 0 1 2 3 4 5 6 7 8 9 10)))
(display (map fib (make-list 0 10)))
(newline)
(display (even-fibs 10))
(newline)

(0 1 1 2 3 5 8 13 21 34 55)
(0 2 8 34)


#### 列の演算

In [3]:
(map square (list 1 2 3 4 5))

(1 4 9 16 25)

In [4]:
; フィルタリング
(define (filter predicate sequence)
  (cond ((null? sequence) '())
        ((predicate (car sequence))(cons (car sequence) (filter predicate (cdr sequence))))
        (else (filter predicate (cdr sequence))))
  )

; 動作確認
(filter odd? (list 1 2 3 4 5))

(1 3 5)

In [5]:
; 集積
(define (accumulate op initial sequence)
  (if (null? sequence) initial
      (op (car sequence) (accumulate op initial (cdr sequence)))))

; 動作確認
(display (accumulate + 0 (list 1 2 3 4 5)))
(newline)
(display (accumulate * 1 (list 1 2 3 4 5)))
(newline)
(display (accumulate cons '() (list 1 2 3 4 5)))
(newline)

15
120
(1 2 3 4 5)


In [6]:
; 整数列の列挙
(define (enumerate-interval low high)
  (if (> low high) '()
      (cons low (enumerate-interval (+ low 1) high))))

(enumerate-interval 2 7)

(2 3 4 5 6 7)

In [7]:
; 木の葉の列挙
(define (enumerate-tree tree)
  (cond ((null? tree) '())
        ((not (pair? tree)) (list tree))
        (else (append (enumerate-tree (car tree)) (enumerate-tree (cdr tree)))))
  )

(enumerate-tree (list 1 (list 2 (list 3 4)) 5))

(1 2 3 4 5)

In [8]:
(define (sum-odd-squares tree)
  (accumulate + 0 (map square (filter odd? (enumerate-tree tree))))
  )

(sum-odd-squares (list 1 (list 2 (list 3 4) 5) (list 6 7)))

84

In [9]:
(define (even-fibs n)
  (accumulate cons '() (filter even? (map fib (enumerate-interval 0 n)))))

(even-fibs 10)

(0 2 8 34)

In [10]:
; フィボナッチ数の2乗の列挙
(define (list-fib-squares n)
  (accumulate cons '() (map square (map fib (enumerate-interval 0 n)))))

(list-fib-squares 10)

(0 1 1 4 9 25 64 169 441 1156 3025)

In [11]:
; 奇数の値の2乗の値の積
(define (product-of-squares-of-odd-elements sequence)
  (accumulate * 1 (map square (filter odd? sequence))))

(product-of-squares-of-odd-elements (list 1 2 3 4 5))

225

salary

#### マップのネスト

$$
\begin{array}
{l|ccccccc}
i & 2 & 3 & 4 & 4 & 5 & 6 & 6 \\
j & 1 & 2 & 1 & 3 & 2 & 1 & 5 \\ \hline
i+j & 3 & 5 & 5 & 7 & 7 & 7 & 11 
\end{array}
$$

In [12]:
(accumulate append '()
            (map (lambda (i)
                   (map (lambda (j) (list i j))
                        (enumerate-interval 1 (- i 1)))
                   )
                 (enumerate-interval 1 6)
                 )
    )

((2 1) (3 1) (3 2) (4 1) (4 2) (4 3) (5 1) (5 2) (5 3) (5 4) (6 1) (6 2) (6 3) (6 4) (6 5))

In [13]:
(enumerate-interval 1 6)

(1 2 3 4 5 6)

In [14]:
(define (flatmap proc seq)
  (accumulate append '() (map proc seq)))

In [15]:
(define (prime-sum? pair)
  (prime? (+ (car pair) (cadr pair))))

In [16]:
(define (make-pair-sum pair)
  (list (car pair) (cadr pair) (+ (car pair) (cadr pair))))

In [17]:
(define (prime-sum-pairs n)
  (map make-pair-sum (filter prime-sum?
                             (flatmap (lambda (i)
                                        (map (lambda (j) (list i j))
                                             (enumerate-interval 1 (- i 1)))
                                        )
                                (enumerate-interval 1 n)
                               )
                        )
       )
  )

In [18]:
(define (smallest-divisor n)
  (find-divisor n 2)
  )
(define (find-divisor n test-divisor)
  (cond ((> (square test-divisor) n) n)
        ((divides? test-divisor n) test-divisor)
        (else (find-divisor n (+ test-divisor 1)))
    )
  )
(define (divides? a b)
  (= (remainder b a) 0)
  )
(define (prime? n)
  (= n (smallest-divisor n))
  )

In [19]:
(prime-sum-pairs 6)

((2 1 3) (3 2 5) (4 1 5) (4 3 7) (5 2 7) (6 1 7) (6 5 11))

In [20]:
; 順列
(define (permutations s)
  (if (null? s) ;集合は空か?
      (list '()) ;空集合を持つ列
      (flatmap (lambda (x) (map (lambda (p) (cons x p))
                                (permutations (remove x s)))) s)))

In [21]:
(define (remove item sequence)
  (filter (lambda (x) (not (= x item))) sequence))

In [22]:
(permutations (list 1 2 3))

((1 2 3) (1 3 2) (2 1 3) (2 3 1) (3 1 2) (3 2 1))